# 案例1.2：城市需水预测 - 教学版

## 学习目标

1. 掌握多种需水预测方法
2. 理解情景分析的思想
3. 学习组合预测技术
4. 能够进行不确定性分析

---

## 1. 环境准备

In [ ]:
import sys
sys.path.insert(0, '../code')
sys.path.insert(0, '../code/examples/case02_water_demand_forecasting')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.quota_method import QuotaForecaster
from src.trend_analysis import TrendForecaster
from src.grey_model import GreyForecaster

plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

print("环境准备完成！")

## 2. 加载历史数据

In [ ]:
# 加载数据
data = pd.read_csv('../code/examples/case02_water_demand_forecasting/data/historical_data.csv')

print(f"数据范围: {data['year'].min()} - {data['year'].max()}")
print(f"数据点数: {len(data)}年")

# 显示数据
data.tail()

## 3. 方法1：定额法预测

公式：Q = 人口 × 人均用水定额

In [ ]:
# 创建预测器
forecaster = QuotaForecaster(
    domestic_quota=260,  # L/人·日
    industrial_quota=42,  # m³/万元
    service_quota=18      # m³/万元
)

# 预测2025年
result_2025 = forecaster.forecast_total(
    population=380,
    industrial_output=1700,
    service_output=2500,
    water_saving_factor=0.90
)

print("定额法预测结果（2025年）:")
print(f"  生活需水: {result_2025['domestic']:.2f} 万m³/d")
print(f"  工业需水: {result_2025['industrial']:.2f} 万m³/d")
print(f"  服务业需水: {result_2025['service']:.2f} 万m³/d")
print(f"  总需水: {result_2025['total']:.2f} 万m³/d")

## 4. 方法2：趋势分析法

In [ ]:
# 线性趋势预测
trend_forecaster = TrendForecaster(method='linear')
trend_forecaster.fit(data['year'].values, data['water_demand'].values)

# 预测2025年
pred_2025 = trend_forecaster.predict(np.array([2025]))[0]

print(f"趋势法预测结果（2025年）:")
print(f"  拟合方程: {trend_forecaster.get_equation()}")
print(f"  R² = {trend_forecaster.score():.4f}")
print(f"  预测值: {pred_2025:.2f} 万m³/d")

## 5. 方法3：灰色预测GM(1,1)

In [ ]:
# 灰色预测
grey_model = GreyForecaster()
grey_model.fit(data['water_demand'].values)

# 预测5步（到2025年）
predictions = grey_model.forecast(5)

# 精度检验
mre, C, grade = grey_model.accuracy_test()

print(f"灰色预测GM(1,1):")
print(f"  模型参数: {grey_model.get_params()['equation']}")
print(f"  平均相对误差: {mre:.4f}")
print(f"  后验差比 C: {C:.4f}")
print(f"  精度等级: {grade}")
print(f"\n  2025年预测: {predictions[0]:.2f} 万m³/d")

## 6. 组合预测

In [ ]:
# 各方法的预测值
quota_pred = result_2025['total']
trend_pred = pred_2025
grey_pred = predictions[0]

# 简单平均
ensemble_pred = (quota_pred + trend_pred + grey_pred) / 3

print("2025年预测结果对比:")
print(f"  定额法: {quota_pred:.2f} 万m³/d")
print(f"  趋势法: {trend_pred:.2f} 万m³/d")
print(f"  灰色法: {grey_pred:.2f} 万m³/d")
print(f"  组合预测: {ensemble_pred:.2f} 万m³/d")

## 7. 思考题

1. 为什么需要使用多种预测方法？
2. 如何确定组合预测的权重？
3. 情景分析的意义是什么？
4. 灰色预测适用于什么情况？

---

## 总结

本案例完成了：
- ✅ 定额法预测
- ✅ 趋势分析法
- ✅ 灰色预测GM(1,1)
- ✅ 组合预测

**关键技能**：
- 多种预测方法对比
- 组合预测思想
- 情景分析方法